# Organize my branch

In [40]:
import numpy as np
#import pandas as pd

#from scipy.integrate import cumtrapz

import gm2
import trfp
from numpy import sqrt

#import doctest
#doctest.testmod(verbose=False)

In [41]:
## Legendre polynomial moments
legendre_coeffs = np.array([[1./2., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # const
                            [0, sqrt(3)/2., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # x
                            [0, 0, sqrt(3)/2., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # y
                            [sqrt(5)/-4., 0, 0, sqrt(5)*3./4., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # x^2
                            [0, 0, 0, 0, 3./2., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  #xy
                            [sqrt(5)/-4., 0, 0, 0, 0, sqrt(5)*3./4., 0, 0, 0, 0, 0, 0, 0, 0, 0],  # y^2
                            [0, sqrt(7)*-3./4., 0, 0, 0, 0, sqrt(7)*5./4., 0, 0, 0, 0, 0, 0, 0, 0],  # x^3
                            [0, 0, sqrt(15)*-1./4., 0, 0, 0, 0, sqrt(15)*3./4., 0, 0, 0, 0, 0, 0, 0],  #x^2 y
                            [0, sqrt(15)*-1./4., 0, 0, 0, 0, 0, 0, sqrt(15)*3./4., 0, 0, 0, 0, 0, 0], # x y^2
                            [0, 0, sqrt(7)*-3./4., 0, 0, 0, 0, 0, 0, sqrt(7)*5./4., 0, 0, 0, 0, 0],  # y^3
                            [9./16., 0, 0, -90./16., 0, 0, 0, 0, 0, 0, 105./16., 0, 0, 0, 0],  # x^4
                            [0, 0, 0, 0, sqrt(21)*-3./4., 0, 0, 0, 0, 0, 0, sqrt(21)*5./4., 0, 0, 0],  # x^3 y
                            [5./8., 0, 0, -15./8., 0, -15./8., 0, 0, 0, 0, 0, 0, 45./8., 0, 0],  # x^2 y^2
                            [0, 0, 0, 0, sqrt(21)*-3./4., 0, 0, 0, 0, 0, 0, 0, 0, sqrt(21)*5./4., 0],  # x y^3
                            [9./16., 0, 0, 0, 0, -90./16., 0, 0, 0, 0, 0, 0, 0, 0, 105./16.]]  # y^4
                          )

## Cylindrical Legendre polynomial coefficients
## Has a weighting term (711.2/4.5 + x) in dot product
## Not precise due to stupid 711.2/4.5 term, could probably precise-ify it

cyl_leg_coeffs = np.array([[0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # const
                           [-0.0001453, 0.06889, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # x
                           [0, 0, 0.06889, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # y
                           [-0.04447, -0.0002251, 0, 0.1334, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # x^2
                           [0, 0, -0.0002517, 0, 0.1193, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # xy
                           [-0.04447, 0, 0, 0, 0, 0.1334, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # y^2
                           [0.0001427, 0.1578, 0, -0.000428, 0, 0, 0.2631, 0, 0, 0, 0, 0, 0, 0, 0],  # x^3
                           [0, 0, -0.07702, 0, -0.0003899, 0, 0, 0.2311, 0, 0, 0, 0, 0, 0, 0],  # x^2 y
                           [0.0001624, 0.07702, 0, 0, 0, 0.0004873, 0, 0, 0.2311, 0, 0, 0, 0, 0, 0],  # x y^2
                           [0, 0, -0.1578, 0, 0, 0, 0, 0, 0, 0.2631, 0, 0, 0, 0, 0],  # y^3
                           [0.04474, 0.0005033, 0, -0.4474, 0, 0, -0.0008389, 0, 0, 0, 0.522, 0, 0, 0, 0],  # x^4
                           [0, 0,  0.0002471, 0, -0.2734, 0, 0, 0.0007414, 0, 0, 0, 0.4557, 0, 0, 0],  # x^3 y
                           [0.04972, 0.0002517, 0, -0.1491, -0.1491, 0, 0, 0, -0.000755, 0, 0, 0, 0.4474, 0, 0],  # x^2 y^2
                           [0, 0, 0.0005766, 0, -0.2734, 0, 0, 0, 0, -0.000961, 0, 0, 0, 0.4557, 0],  # x y^3
                           [0.04474, 0, 0, 0, 0, -0.4474, 0, 0, 0, 0, 0, 0, 0, 0, 0.522]]  # y^4
                         )
def legendre_moment(l, x, y, cyl=True):
    num = len(x)
    terms = np.array([np.ones(num), x, y, x**2, x*y, y**2, x**3, x**2*y, x*y**2,
                      y**3, x**4, x**3*y, x**2*y**2, x*y**3, y**4])
   
    values = np.empty([num,15])
    for ii in range(15): values[:,ii] = terms[ii]
       
    if cyl: coeffs = cyl_leg_coeffs
    else: coeffs = legendre_coeffs
       
    output = np.matmul(values, coeffs[l])
    output[np.abs(output)<1.e-12] = 0
   
    return output

In [42]:
# First Defined Functions:
# for loop to make as high n and l as desired.
# use trfp.Station id as numbering system ([station, probe, val])

In [63]:
#Make a class similar to the Geometry file, made for dropping probes/ drop with 2 options:geometrical or number
def legendre_moment3d(LNS, x, y, phi, cyl = True):
    """ Computes 3D Legendre Moments.
    
    Takes regular Legendre Moment as developed by Alec Tewsley-Booth and 
    adds a cos(n \phi) or sin(n\phi term). L corresponds to the Legendre
    Moment order (L0,...,Lm), N corresponds to the azimuthal order or
    periodicity (0 ,..., n). Cosine is "normal" while sine is "skew". Note
    that are is no zeroth order skew Legendre Moments since Sin(0) = 0.
    
    Args:
    LNS: a (3, X) numpy array containing the moments to be calculated;
         X is any length > 1 this array is usually provided by the
         FlattenMoment function.
    
    x:   a 1-d array containing the x-positions of the P points where the
         Legendre Moments are to be calculated.
         
    y:   a 1-d array containing the y-positions of the P points where the
         Legendre Moments are to be calculated.
         
    phi: a 1-d array containing the phi-positions (in radians) of the P
         points where the Legendre Moments are to be calculated.
         
    cyl: argument that if True calculates cylindrical Legendre Moments. If
         False, calculates cartesian Legendre Moments.
    
    """
    s = LNS[2]
    if s == 1:
        return legendre_moment(LNS[0],x,y,cyl)*np.sin(LNS[1]*phi)
    elif s == 0:
        return legendre_moment(LNS[0],x,y,cyl)*np.cos(LNS[1]*phi)
    
class RingGeometry:
    """
    A class that stores the x, y and phi positions of the probes to be used.
    In the case of fixed probes, it also containes which probes are function-
    al. Additional functions to simplify other functions are also here. x, y,
    phi and p have the same format as trfp.STATION_PROBE_ID for fixed probes.
    """
    def __init__(self, x, y, phi, p):
        """
        Initialized the class, not much to say here except that p is an array
        of 0s and 1s where 0 means that such a probe is non-functional and
        1 means it is functional
        """
        self.xpos = x
        self.ypos = y
        self.phis = phi
        self.probes = p
    
    def IsReduced(self):
        """
        Checks to see whether a geometry instance is reduced. If a geometry
        class is reduced then all probes are functional. Otherwise it is not
        reduced.
        
        These are some tests:
        >>> x = [np.arange(5)]
        >>> y = [np.arange(5)]
        >>> phi = [np.linspace(0, 360, 5)]
        >>> p = [np.zeros(5) +1]
        >>> geom = RingGeometry(x ,y ,phi ,p)
        >>> geom.IsReduced()
        True
        
        >>> p = [np.array([0,1,1,1,1])]
        >>> geom = RingGeometry(x ,y ,phi ,p)
        >>> geom.IsReduced()
        False
        """
        isReduced = True
        
        for i in range(len(self.probes)):
            for j in range(len(self.probes[i])):
                if self.probes[i][j] != 1:
                    return False
        return isReduced
    
    def getBadProbes(self):
        badProbes = []
        for i in range(len(self.probes)):
            badProbesInThisStation = np.array([])
            for j in range(len(self.probes[i])):
                if not self.probes[i][j]:
                    badProbesInThisStation = np.append(badProbesInThisStation,j)
                
            badProbes.append(badProbesInThisStation.astype(int))
        return badProbes
    
    def getNumberOfBadProbes(self):
        '''
        
        '''
        N = 0
        badProbes = self.getBadProbes()
        for i in range(len(badProbes)):
            for j in range(len(badProbes[i])):
                N+=1
        return N
                
    
    def getNumberOfProbes(self):
        N = 0
        for i in range(len(self.probes)):
            N+= len(self.probes[i])
        return N

def Drop3dPos(geom): #can make XYPhiProbeArray
    badProbes = geom.getBadProbes()
    newx = []
    newy = []
    newPhis = []
    newProbes = []
    
    for i in range(len(geom.probes)):
        if len(badProbes[i]) == len(geom.probes[i]):
            continue
        PROBES = geom.probes[i].copy()
        X = geom.xpos[i].copy()
        Y = geom.ypos[i].copy()
        PHI = geom.phis[i].copy()
        
        newx.append(np.delete(X, badProbes[i]))
        newy.append(np.delete(Y, badProbes[i]))
        newPhis.append(np.delete(PHI, badProbes[i]))
        newProbes.append(np.delete(PROBES, badProbes[i]))
    ReducedGeometry = RingGeometry(newx, newy, newPhis, newProbes)
    return ReducedGeometry, badProbes

# Caps is not right
def FlattenMoments(lcap, ncap): #First lcap Lmoments and first ncap Az. moments (n=0 counts as 1 too)
    Ls = np.tile(np.repeat(np.arange(lcap),2), ncap)
    Ns = np.sort(np.repeat(np.arange(ncap), lcap*2))
    Ss = np.tile([0,1], lcap*ncap)
    return np.delete(Ls,np.arange(lcap)*2 +1),\
           np.delete(Ns,np.arange(lcap)*2 +1),\
           np.delete(Ss,np.arange(lcap)*2 +1)

def CreateLNSHeader(lcap,ncap): #n1s,l3
    Ls, Ns, Ss = FlattenMoments(lcap, ncap)
    Header = []
    for i in range(len(Ls)):
        col = 'N' + str(Ns[i])
        if Ss[i] == 1:
            col = col + 's,'
        else:
            col = col + 'n,'
        col = col + 'l' + str(Ls[i])
        Header.append(col)
    return Header

def InvMultipole(MomentFxn, geom, lcap, ncap, cylindrical = True):
    Xs = np.concatenate(geom.xpos)
    Ys = np.concatenate(geom.ypos)
    Phis = np.concatenate(geom.phis)
    
    LNS = np.zeros((2*lcap*ncap - lcap, 3), dtype = int)
    LNS[:,0], LNS[:,1], LNS[:,2] = FlattenMoments(lcap, ncap)
    
    
    if (len(LNS)>geom.getNumberOfProbes()):
        print("Not enough probes for the Moments, overfitting...")
        
    MULTS = np.array([MomentFxn(LNS[i], Xs, Ys, Phis, cyl = cylindrical)\
                                for i in range(len(LNS))])
    
    MULTS[np.abs(MULTS) < 1.0e-9] = 0
    
    INV_MULT = np.linalg.pinv(np.transpose(MULTS))
    INV_MULT[np.abs(INV_MULT) < 1.0e-9] = 0
    
    I = np.dot(INV_MULT, np.transpose(MULTS))
    I[np.abs(I) < 1.0e-9] = 0
    print(np.sum(np.abs(I)) - min(len(I), len(I[0])))
    
    return INV_MULT
    
    
#put this above ProbeDropsL Also, must make a copy of badProbes
def Theta3dL(MomentFxn, geom, badProbes, lcap, ncap, cylindrical = True):
    lenOrigProbes = 378
    INV_MULT = InvMultipole(MomentFxn, geom, lcap, ncap, cylindrical)
    
    inv_mult = np.zeros((len(INV_MULT), lenOrigProbes))
    k = 0
    l = 0 #This counts how many iterations have been done in the loop
    if geom.getNumberOfProbes() < lenOrigProbes:
        for i in range(len(trfp.STATION_PROBE_ID)):
            for j in range(len(trfp.STATION_PROBE_ID[i])):
                if (len(badProbes[i]) > 0) and (j == badProbes[i][0]):
                    badProbes[i] = np.delete(badProbes[i], [0]) #"skip"
                    l+=1
                else:
                    inv_mult[:,l] = INV_MULT[:,k]
                    k+=1
                    l+=1
        return inv_mult
    else:
        return INV_MULT

In [65]:
Test = np.zeros((4,5))
test_df = pd.DataFrame(Test)
print(test_df)

     0    1    2    3    4
0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0


In [66]:
x = [np.arange(5)]
y = [np.arange(5)]
phi = [np.arange(5)]
p = [np.array([1,1,1,0,1])]
geom = RingGeometry(x ,y ,phi ,p)
geom.IsReduced()

False

# Fxn that takes trolley run and puts it into fp format to run through same fxns.
(4000 st of 17 probes)

In [67]:
#CreateLNSHeader(lcap=3, ncap= 2)

In [68]:
def ProbeDrops3dL(MomentFxn, geom, lcap, ncap, cylindrical = True):
    reducedGeometry, badProbes = Drop3dPos(geom)
    
    return Theta3dL(MomentFxn, reducedGeometry, badProbes, lcap, ncap, cylindrical=cylindrical)

In [69]:
Phis = np.array(trfp.STATION_BARCODE_PHI) # Barcode phi

#In radian
Phis = Phis*2*np.pi/360

probes = []
xpos = []
ypos = []
phi = []

for i in range(len(trfp.STATION_PROBE_ID)):
    numProbes = len(trfp.STATION_PROBE_ID[i])
    probes.append(np.zeros(numProbes)+1)
    if numProbes == 4:
        if (i == 37) or (i == 39):
            xpos.append(trfp.FP4_X_ST37_ST39)
        
        elif (i == 41):
            xpos.append(trfp.FP4_X_ST41)
        
        else:
            xpos.append(trfp.FP4_X)
            
        ypos.append(trfp.FP4_Y)
    if numProbes == 6:
        xpos.append(trfp.FP6_X)
        ypos.append(trfp.FP6_Y)
    
    phi.append(np.zeros(numProbes)+ Phis[i])

In [70]:
# Make all stations into 4-probe stations
#for i in range(len(probes)):
#    if len(probes[i]) == 6:
#        probes[i] = np.array([0,1,1,0,1,1])

In [71]:
test = RingGeometry(xpos, ypos, phi, probes) #n1s,l3

# Plug back coefficients into 3dLeg to see residuals.

In [75]:
for i in range(50):
    print('*******************')
    print("n = " + str(i+1))
    M = ProbeDrops3dL(legendre_moment3d, test, lcap=5, ncap= i+1) #Up to 36 when we cap up to 3 (Not enough 6-probe stations)

*******************
n = 1
-1.7763568394002505e-15
*******************
n = 2
3.552713678800501e-15
*******************
n = 3
0.0
*******************
n = 4
7.105427357601002e-15
*******************
n = 5
-7.105427357601002e-15
*******************
n = 6
0.0
*******************
n = 7
0.0
*******************
n = 8
0.0
*******************
n = 9
0.0
*******************
n = 10
0.0
*******************
n = 11
0.0
*******************
n = 12
0.0
*******************
n = 13
0.0
*******************
n = 14
0.0
*******************
n = 15
0.0
*******************
n = 16
0.0
*******************
n = 17
0.0
*******************
n = 18
0.0
*******************
n = 19
-5.684341886080802e-14
*******************
n = 20
2.842170943040401e-14
*******************
n = 21
0.0
*******************
n = 22
0.0
*******************
n = 23
-5.684341886080802e-14
*******************
n = 24
1.9895196601282805e-13
*******************
n = 25
19.14439893853512
*******************
n = 26
53.74014375428561
*******************
n = 2

In [11]:
def ThetaL(Multipole, geom, badProbes, tpMomentCap, fpMomentCap, cylindrical=True): 

    lenOrigProbes = len(geom.probes) + len(badProbes)
    
    if (lenOrigProbes > 6):
        momentCap = tpMomentCap
    else:
        momentCap = fpMomentCap
    
    if (len(geom.probes) < momentCap):
        print("Moment cap is higher for the number of functional probes")
    
    _MULTS_FP_N = np.array([Multipole(i, geom.xpos, geom.ypos, cyl=cylindrical)\
                                for i in range(momentCap)])
        
    _MULTS_FP_N[np.abs(_MULTS_FP_N) < 1.0e-9] = 0
    
    INV_MULT = np.linalg.pinv(np.transpose(_MULTS_FP_N))
    INV_MULT[np.abs(INV_MULT) < 1.0e-9] = 0
    
    I = np.dot(INV_MULT,np.transpose(_MULTS_FP_N))
    I[np.abs(I) < 1.0e-9] =0
    #print(I)
    #Add the columns of 0
    inv_mult = np.zeros((len(INV_MULT), lenOrigProbes))
    j = 0
    if len(badProbes) > 0:
        badProbesCopy = badProbes.copy()
        for i in range(lenOrigProbes):
            if i != badProbesCopy[0]:
                inv_mult[:,i] = INV_MULT[:,j]
                j+=1
            elif (i == badProbesCopy[0]) and (len(badProbesCopy) > 1):
                badProbesCopy = np.delete(badProbesCopy, 0)
                  
        #Add rows at the end, only the first moments can be calculated
        for i in range(lenOrigProbes - momentCap):
            inv_mult = np.concatenate((inv_mult, np.zeros((1,lenOrigProbes))))
        return(inv_mult)
    else:
        for i in range(lenOrigProbes - momentCap):
            INV_MULT = np.concatenate((INV_MULT, np.zeros((1,lenOrigProbes))))
            
        return(INV_MULT)
    
def ProbeDropsL(geom, tpMomentCap = 14, fpMomentCap = 5, cylindrical = True):
    reducedGeometry, badProbes = DropPos(geom)
    
    return ThetaL(legendre_moment, reducedGeometry, badProbes, tpMomentCap, fpMomentCap, cylindrical=cylindrical)

In [12]:
# Probe Geometry.
class Geometry:
    def __init__(self, x, y, p):
        self.xpos = x
        self.ypos = y
        self.probes = p.astype('int')
        if len(self.xpos) != len(self.ypos):
            print("In Geometry length of xpos and ypos are not equal")
        elif len(self.ypos) != len(self.probes):
            print("In Geometry length of probes and ypos are not equal")
    def IsReduced(self):
        isReduced = True
        for i in self.probes:
            if i == 0:
                isReduced = i
        return isReduced
    
def DropPos(geom):
    badProbes = np.array([])
    for i in range(len(geom.probes)):
        if not geom.probes[i]:
            badProbes = np.append(badProbes, i)
    badProbes = badProbes.astype(int)
    X = geom.xpos.copy()
    Y = geom.ypos.copy()
    px_new = np.delete(X, badProbes) # Only the x pos of working probes stay
    py_new = np.delete(Y, badProbes) # Same for y
    PROBES_NEW = np.delete(geom.probes, badProbes) #Now this is an array of 1s.
    reducedGeometry = Geometry(px_new, py_new, PROBES_NEW)
    return reducedGeometry, badProbes

def ThetaL(Multipole, geom, badProbes, tpMomentCap, fpMomentCap, cylindrical=True): 

    lenOrigProbes = len(geom.probes) + len(badProbes)
    
    if (lenOrigProbes > 6):
        momentCap = tpMomentCap
    else:
        momentCap = fpMomentCap
    
    if (len(geom.probes) < momentCap):
        print("Moment cap is higher for the number of functional probes")
    
    _MULTS_FP_N = np.array([Multipole(i, geom.xpos, geom.ypos, cyl=cylindrical)\
                                for i in range(momentCap)])
        
    _MULTS_FP_N[np.abs(_MULTS_FP_N) < 1.0e-9] = 0
    
    INV_MULT = np.linalg.pinv(np.transpose(_MULTS_FP_N))
    INV_MULT[np.abs(INV_MULT) < 1.0e-9] = 0
    
    I = np.dot(INV_MULT,np.transpose(_MULTS_FP_N))
    I[np.abs(I) < 1.0e-9] =0
    #print(I)
    #Add the columns of 0
    inv_mult = np.zeros((len(INV_MULT), lenOrigProbes))
    j = 0
    if len(badProbes) > 0:
        badProbesCopy = badProbes.copy()
        for i in range(lenOrigProbes):
            if i != badProbesCopy[0]:
                inv_mult[:,i] = INV_MULT[:,j]
                j+=1
            elif (i == badProbesCopy[0]) and (len(badProbesCopy) > 1):
                badProbesCopy = np.delete(badProbesCopy, 0)
                  
        #Add rows at the end, only the first moments can be calculated
        for i in range(lenOrigProbes - momentCap):
            inv_mult = np.concatenate((inv_mult, np.zeros((1,lenOrigProbes))))
        return(inv_mult)
    else:
        for i in range(lenOrigProbes - momentCap):
            INV_MULT = np.concatenate((INV_MULT, np.zeros((1,lenOrigProbes))))
            
        return(INV_MULT)
    
def ProbeDropsL(geom, tpMomentCap = 14, fpMomentCap = 5, cylindrical = True):
    reducedGeometry, badProbes = DropPos(geom)
    
    return ThetaL(legendre_moment, reducedGeometry, badProbes, tpMomentCap, fpMomentCap, cylindrical=cylindrical)